In [37]:
from aocd import get_data

puzzle_input = get_data(day=9, year=2015)

In [20]:
puzzle_input.split("\n")

['Tristram to AlphaCentauri = 34',
 'Tristram to Snowdin = 100',
 'Tristram to Tambi = 63',
 'Tristram to Faerun = 108',
 'Tristram to Norrath = 111',
 'Tristram to Straylight = 89',
 'Tristram to Arbre = 132',
 'AlphaCentauri to Snowdin = 4',
 'AlphaCentauri to Tambi = 79',
 'AlphaCentauri to Faerun = 44',
 'AlphaCentauri to Norrath = 147',
 'AlphaCentauri to Straylight = 133',
 'AlphaCentauri to Arbre = 74',
 'Snowdin to Tambi = 105',
 'Snowdin to Faerun = 95',
 'Snowdin to Norrath = 48',
 'Snowdin to Straylight = 88',
 'Snowdin to Arbre = 7',
 'Tambi to Faerun = 68',
 'Tambi to Norrath = 134',
 'Tambi to Straylight = 107',
 'Tambi to Arbre = 40',
 'Faerun to Norrath = 11',
 'Faerun to Straylight = 66',
 'Faerun to Arbre = 144',
 'Norrath to Straylight = 115',
 'Norrath to Arbre = 135',
 'Straylight to Arbre = 127']

In [26]:
import re

dist_pattern = re.compile(r"(\w+) to (\w+) = (\d+)")

def construct_dist_matrix(distances):
    if isinstance(distances, str):
        distances = distances.split("\n")
    
    villes = set()
    distances_matrix = {}

    for dist in distances:
        v1, v2, d = dist_pattern.match(dist).groups()

        villes.add(v1)
        villes.add(v2)

        distances_matrix[v1, v2] = int(d)
        distances_matrix[v2, v1] = int(d)

    return villes, distances_matrix

villes_example, dist_example = construct_dist_matrix("""London to Dublin = 464
London to Belfast = 518
Dublin to Belfast = 141""")

In [39]:
villes_puzzle, dist_puzzle = construct_dist_matrix(puzzle_input)
villes_puzzle, dist_puzzle

({'AlphaCentauri',
  'Arbre',
  'Faerun',
  'Norrath',
  'Snowdin',
  'Straylight',
  'Tambi',
  'Tristram'},
 {('Tristram', 'AlphaCentauri'): 34,
  ('AlphaCentauri', 'Tristram'): 34,
  ('Tristram', 'Snowdin'): 100,
  ('Snowdin', 'Tristram'): 100,
  ('Tristram', 'Tambi'): 63,
  ('Tambi', 'Tristram'): 63,
  ('Tristram', 'Faerun'): 108,
  ('Faerun', 'Tristram'): 108,
  ('Tristram', 'Norrath'): 111,
  ('Norrath', 'Tristram'): 111,
  ('Tristram', 'Straylight'): 89,
  ('Straylight', 'Tristram'): 89,
  ('Tristram', 'Arbre'): 132,
  ('Arbre', 'Tristram'): 132,
  ('AlphaCentauri', 'Snowdin'): 4,
  ('Snowdin', 'AlphaCentauri'): 4,
  ('AlphaCentauri', 'Tambi'): 79,
  ('Tambi', 'AlphaCentauri'): 79,
  ('AlphaCentauri', 'Faerun'): 44,
  ('Faerun', 'AlphaCentauri'): 44,
  ('AlphaCentauri', 'Norrath'): 147,
  ('Norrath', 'AlphaCentauri'): 147,
  ('AlphaCentauri', 'Straylight'): 133,
  ('Straylight', 'AlphaCentauri'): 133,
  ('AlphaCentauri', 'Arbre'): 74,
  ('Arbre', 'AlphaCentauri'): 74,
  ('Snowdi

In [40]:
import heapq

def best_road(villes, distances_matrix):

    # Initialisation
    distances = []

    villes_non_visites = tuple(villes)
    for i, v in enumerate(villes_non_visites):
        heapq.heappush(distances, (0, v, villes_non_visites[:i] + villes_non_visites[i+1:]))

    while True:
        dist, origine, villes_non_visites = heapq.heappop(distances)

        if not villes_non_visites:
            return dist

        for i, destination in enumerate(villes_non_visites):
            heapq.heappush(
                distances, 
                (   
                    dist + distances_matrix[(origine, destination)], 
                    destination, 
                    villes_non_visites[:i] + villes_non_visites[i+1:]
                )
            )


best_road(villes_example, dist_example)

605

In [41]:
best_road(villes_puzzle, dist_puzzle)

251

In [47]:
# On peut pas généraliser la première méthode pour un calcul efficace dans le cas du chemin le plus long ? 
#
# La condition d'arrêt est que la liste des villes restantes est vide. On est assuré d'avoir le chemin le plus
# court dans la première partie car un la distance est croissante et que l'on prend toujours la plus petite distance
# mais dans le cas du chemin le plus long l'astuce ne fonctionne plus.

def worst_road(villes, distances_matrix):

    # Initialisation
    distances = []

    villes_non_visites = tuple(villes)
    for i, v in enumerate(villes_non_visites):
        heapq.heappush(distances, (1, 0, (v, ), villes_non_visites[:i] + villes_non_visites[i+1:]))

    while True:
        len_chemin, dist, chemin, villes_non_visites = heapq.heappop(distances)

        if not villes_non_visites:
            return (-dist, chemin)

        for i, destination in enumerate(villes_non_visites):
            heapq.heappush(
                distances, 
                (   
                    len_chemin + 1,
                    dist - distances_matrix[(chemin[-1], destination)], 
                    chemin + (destination,), 
                    villes_non_visites[:i] + villes_non_visites[i+1:]
                )
            )

worst_road(villes_example, dist_example)

(982, ('Belfast', 'London', 'Dublin'))

In [48]:
worst_road(villes_puzzle, dist_puzzle)

(898,
 ('Snowdin',
  'Tambi',
  'Norrath',
  'AlphaCentauri',
  'Straylight',
  'Arbre',
  'Faerun',
  'Tristram'))

In [43]:
from itertools import permutations

def worst_worst_road(villes, distances_matrix):
    max_p = 0
    argmax_p = None
    for p in permutations(villes):
        dist = sum(map(lambda x, y : distances_matrix[(x, y)], p, p[1:]))
        if dist > max_p:
            max_p, argmax_p = dist, p
    return max_p, argmax_p

worst_worst_road(villes_example, dist_example)

(982, ('Dublin', 'London', 'Belfast'))

In [44]:
worst_worst_road(villes_puzzle, dist_puzzle)

(898,
 ('Snowdin',
  'Tambi',
  'Norrath',
  'AlphaCentauri',
  'Straylight',
  'Arbre',
  'Faerun',
  'Tristram'))